In [2]:
%load_ext autoreload
%autoreload 2


import modish
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline


folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities_motifs'

import flotilla
flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'

study = flotilla.embark('singlecell_pnm_figure2_modalities', flotilla_dir=flotilla_dir)
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=20))

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


2015-12-02 07:51:26	Reading datapackage from /home/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities/datapackage.json
2015-12-02 07:51:26	Parsing datapackage to create a Study object
2015-12-02 07:52:14	Initializing Study
2015-12-02 07:52:14	Initializing Predictor configuration manager for Study
2015-12-02 07:52:14	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2015-12-02 07:52:14	Added ExtraTreesClassifier to default predictors
2015-12-02 07:52:14	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2015-12-02 07:52:14	Added ExtraTreesRegressor to default predictors
2015-12-02 07:52:14	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2015-12-02 07:52:14	Added GradientBoostingClassifier to default predictors
2015-12-02 07:52:14	Predictor GradientBoostingRegressor is of type <class 'sklearn.ensemble.gradient_boost

### Make BED files of each modality within each celltype

Use the other modalities, within that celltype, as background.

  - What properties are unique to events that are bimodal in iPSC, rather than ~0 or ~1?
    - Foreground: Bimodal events in iPSC
    - Background: All other events in iPSC

In [3]:
# %%time 

# bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities'
# ! mkdir $bed_folder
# import pybedtools

# DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

# exon_bedfile = '{}/exon2.bed'.format(DIR)
# exon_bed = pybedtools.BedTool(exon_bedfile)



# for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
#     background_events = set(phenotype_df.event_name)
#     for modality, modality_df in phenotype_df.groupby('modality'):
#         event_names = set(modality_df.event_name)
#         format_args = bed_folder, phenotype, modality
#         filtered_bed = exon_bed.filter(lambda x: x.name in event_names)
#         filtered_filename = '{}/exon2_{}_{}.bed'.format(*format_args)
#         filtered_bed.saveas(filtered_filename)

#         background_bed = exon_bed.filter(lambda x: x.name not in event_names and x.name in background_events)
#         background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
#         background_bed.saveas(background_filename)

#         ! wc -l $filtered_filename
#         ! wc -l $background_filename

In [54]:
import os

homer_dir = '{}/homer'.format(bed_folder)
!mkdir -p $homer_dir

DIRECTIONS = 'upstream', 'downstream'

def construct_homer_command(foreground_filename, background_filename, homer_flags, findMotifsGenome='findMotifsGenome.pl', 
                            out_dir_prefix=None,
                            force=False):
    if out_dir_prefix is None:
        out_dir = foreground_filename.replace('.bed', '')
    else:
        out_dir = '{}/{}'.format(out_dir_prefix, os.path.basename(foreground_filename).replace('.bed', ''))
    command = '{} {} hg19 {} -bg {} {}'.format(
        findMotifsGenome, foreground_filename, out_dir, background_filename, homer_flags)
    return command

def unique_regions(bed):
    return pybedtools.BedTool(list(set(x for x in bed)))

def get_flanking_intron(bed, direction, genome, nt):
    if direction == 'downstream':
        intron = bed.flank(l=0, r=nt, s=True, genome=genome)
    elif direction == 'upstream':
        intron = bed.flank(l=nt, r=0, s=True, genome=genome)
    else:
        raise ValueError('Only "downstream" and "upstream" are accepted '
                         'direction values, not "{}"'.format(direction))

    # Saved every exon that was exactly upstream or downstream of a junction,
    # So when taking the flanking sequence, there's a lot of repetition
    intron = unique_regions(intron)
    return intron


bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities'
! mkdir $bed_folder
import pybedtools

DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

exon_bedfile = '{}/exon2.bed'.format(DIR)
exon_bed = pybedtools.BedTool(exon_bedfile)

commands = []

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6 -mset vertebrates -mis 0 -p {} -noweight'.format(n_processors)


primate_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPrimates.bed'
primate = pybedtools.BedTool(primate_filename)
placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
placental = pybedtools.BedTool(placental_filename)
conserved_regions = {'primate': primate, 'placental': placental}

genome = 'hg19'

nt = 400

class ForegroundBackgroundPair(object):

    def __init__(self, foreground, background):
        self.foreground = foreground
        self.background = background
        self.beds = {'foreground': self.foreground,
                     'background': self.background}

    def __repr__(self):
        s = 'ForegroundBackgroundPair:\nForeground: {} ({}) entries' \
            '\nBackground: {} ({} entries)'.format(
                self.foreground.fn, len(self.foreground),
                self.background.fn, len(self.background))
        return s

    def __str__(self):
        return self.__repr__()

    @staticmethod
    def _prefix(bed, x_ground):
        return bed.fn.replace(x_ground, '').replace('.bed', '').rstrip('_')

    def intersect(self, other, other_name):
        intersections = {}
        for x_ground, bed in self.beds.items():
            prefix = self._prefix(bed, x_ground)
            intersected_filename = '{prefix}_{other_name}_{x_ground}.bed'.format(
                prefix=prefix, other_name=other_name, x_ground=x_ground)
            bed.intersect(other).saveas(intersected_filename)
            intersected = pybedtools.BedTool(intersected_filename)
            intersections[x_ground] = intersected
        return ForegroundBackgroundPair(**intersections)

    def flanking_intron(self, direction, genome, nt):
        introns = {}
        for x_ground, bed in self.beds.items():
            prefix = self._prefix(bed, x_ground)
            intron_filename = '{prefix}_{direction}{nt}_{name}.bed'.format(
                prefix=prefix, direction=direction, nt=nt, name=x_ground)
            intron = get_flanking_intron(bed, direction, genome, nt)
            intron.saveas(intron_filename)
            intron = pybedtools.BedTool(intron_filename)

            introns[x_ground] = intron
        return ForegroundBackgroundPair(**introns)


for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
    background_events = set(phenotype_df.event_name)
    for modality, modality_df in phenotype_df.groupby('modality'):
        event_names = set(modality_df.event_name)
        format_args = bed_folder, phenotype, modality
        foreground_bed = exon_bed.filter(lambda x: x.name in event_names)
        foreground_filename = '{}/exon2_{}_{}_foreground.bed'.format(*format_args)
        foreground = pybedtools.BedTool(foreground_bed.saveas(foreground_filename))

        background_bed = exon_bed.filter(lambda x: x.name not in event_names and x.name in background_events)
        background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
        background = pybedtools.BedTool(background_bed.saveas(background_filename))

#         ! wc -l $foreground_filename
#         ! wc -l $background_filename
        
        pair = ForegroundBackgroundPair(foreground, background)
        print '\n', pair 
        
        command = construct_homer_command(foreground.fn, background.fn, homer_flags, findMotifsGenome, out_dir_prefix=homer_dir)
        if command is not None:
            commands.append(command)
        
        beds = {'background': background_bed, 'foreground': foreground_bed}
        
        for clade, conservation_bed in conserved_regions.items():
            conserved_pair = pair.intersect(conservation_bed, clade)
#             conserved_beds = {}
#             conserved_filenames = {}
#             for name, bed in beds.items():
#                 conserved = bed.intersect(conservation_bed)
#                 conserved_filename = '{}/exon2_{}_{}_{clade}_{name}.bed'.format(*format_args, clade=clade, name=name)
#                 conserved_beds[name] = conserved
#                 conserved_filenames[name] = conserved_filename
            print '\n', conserved_pair
            command = construct_homer_command(conserved_pair.foreground.fn, 
                                              conserved_pair.background.fn, 
                                              homer_flags, findMotifsGenome, out_dir_prefix=homer_dir)
            if command is not None:
                commands.append(command)

        
        for direction in DIRECTIONS:
            intron_pair = pair.flanking_intron('downstream', 'hg19', 400)
#             direction_beds = {}
#             direction_filenames = {}
#             for name, bed in beds.items():
#                 intron = get_flanking_intron(bed, direction, genome, nt)
#                 intron_filename = '{}/exon2_{}_{}_{direction}{nt}_{name}.bed'.format(
#                     *format_args, direction=direction, nt=nt, name=name)
#                 direction_beds[name] = intron
#                 direction_filenames[name] = intron_filename
            print '\n', intron_pair
            command = construct_homer_command(intron_pair.foreground.fn, 
                                              intron_pair.background.fn, 
                                              homer_flags, findMotifsGenome, out_dir_prefix=homer_dir)
            if command is not None:
                commands.append(command)

            for clade, conservation_bed in conserved_regions.items():
                conserved_introns = intron_pair.intersect(conservation_bed, clade)
#                 conserved_beds = {}
#                 conserved_filenames = {}
#                 for name, bed in beds.items():
#                     conserved = bed.intersect(conservation_bed)
#                     conserved_filename = '{}/exon2_{}_{}_{direction}{nt}_{clade}_{name}.bed'.format(
#                         *format_args, direction=direction, nt=nt, clade=clade, name=name)
#                     conserved_beds[name] = conserved
#                     conserved_filenames[name] = conserved_filename
                print '\n', conserved_introns
                command = construct_homer_command(conserved_introns.foreground.fn,
                                                  conserved_introns.background.fn, 
                                                  homer_flags, findMotifsGenome, out_dir_prefix=homer_dir)
                if command is not None:
                    commands.append(command)
                

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities': File exists
ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_foreground.bed (390) entries
Background: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_background.bed (2206 entries)
ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_placental_foreground.bed (599) entries
Background: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_placental_background.bed (3763 entries)
ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_primate_foreground.bed (297) entries
Background: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_primate_background.bed (1650 entries)
ForegroundBackgroundPair:
F

In [55]:
commands[1]

'/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_placental_foreground.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/homer/exon2_MN_bimodal_placental_foreground -bg /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_MN_bimodal_placental_background.bed -rna -len 4,5,6 -mset vertebrates -mis 0 -p 4 -noweight'

In [56]:
from gscripts.qtools import Submitter

jobname = 'homer'
sub = Submitter(commands, jobname, walltime='0:30:00', queue='home-scrm',
               ppn=n_processors, write_and_submit=True, array=True,
               out_filename='{}/{}.out'.format(homer_dir, jobname),
               err_filename='{}/{}.err'.format(homer_dir, jobname),)

running 108 tasks as an array-job.
job ID: 3822542


In [50]:
len(commands)

108

In [12]:
background = pybedtools.BedTool(background_filename)

In [15]:
asdf = pybedtools.BedTool(background.saveas('asdf.bed'))

In [16]:
asdf.fn

'asdf.bed'

In [38]:
import pybedtools




foreground = pybedtools.BedTool(foreground_filename)
background = pybedtools.BedTool(background_filename)

pair = ForegroundBackgroundPair(foreground, background)
pair


ForegroundBackgroundPair:
Foreground: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_iPSC_~1.bed
Background: /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_iPSC_~1_background.bed

In [39]:
clade

'primate'

In [40]:
conserved_pair = pair.intersect(conservation_bed, clade)
conserved_pair

In [41]:
conserved_pair.foreground.fn

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_iPSC_~1_primate_foreground.bed'

In [42]:
conserved_pair.background.fn

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_iPSC_~1_primate_background.bed'

In [44]:
intron_pair = pair.flanking_intron('downstream', 'hg19', 400)

In [45]:
intron_pair.background.fn

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_iPSC_~1_downstream400_background.bed'

In [46]:
intron_pair.foreground.fn

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_iPSC_~1_downstream400_foreground.bed'

In [28]:
%pdbasd

ERROR: Line magic function `%pdbasd` not found.


In [7]:
get_flanking_intron(bed, direction, genome, nt)

ValueError: Genome file "%s" does not exist

> /home/obotvinnik/anaconda/lib/python2.7/site-packages/pybedtools/bedtool.py(1278)check_genome()
   1277         if not os.path.exists(kwargs['g']):
-> 1278             raise ValueError('Genome file "%s" does not exist')
   1279 

ipdb> dir()
['kwargs', 'self']
ipdb> self
<generator object <genexpr> at 0x2b38d784c4b0>
ipdb> kwargs
{'i': <generator object <genexpr> at 0x2b38d784c4b0>, 's': True, 'r': 0, 'l': 400, 'g': 'hg19'}
ipdb> q


  - What are properties that are unique to bimodal events?
    - Foreground: any event that has ever been bimodal
    - Background: All events that have never been bimodal

In [3]:
%%time 

bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities'
! mkdir $bed_folder
import pybedtools

DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

exon_bedfile = '{}/exon2.bed'.format(DIR)
exon_bed = pybedtools.BedTool(exon_bedfile)

all_events = set(study.supplemental.modalities_tidy.event_name)

for modality, modality_df in study.supplemental.modalities_tidy.groupby('modality'):
    event_names = set(modality_df.event_name)
    background_events = all_events - event_names
    
    format_args = bed_folder, modality
    filtered_bed = exon_bed.filter(lambda x: x.name in event_names)
    filtered_filename = '{}/exon2_{}.bed'.format(*format_args)
    filtered_bed.saveas(filtered_filename)

    background_bed = exon_bed.filter(lambda x: x.name not in event_names and x.name in background_events)
    background_filename = '{}/exon2_{}_background.bed'.format(*format_args)
    background_bed.saveas(background_filename)

    ! wc -l $filtered_filename
    ! wc -l $background_filename

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities': File exists
2155 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_bimodal.bed
5970 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_bimodal_background.bed
1 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_middle.bed
8124 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_middle_background.bed
9 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_multimodal.bed
8116 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_multimodal_background.bed
1770 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0.bed
6355 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_background.bed
4945 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~1.bed
3180 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_

In [4]:
mkdir /home/obotvinnik/projects/singlecell_pnms/analysis/homer/

mkdir: cannot create directory `/home/obotvinnik/projects/singlecell_pnms/analysis/homer/': File exists


## HOMER enrichment


In [5]:
homer_dir = '{}/homer'.format(bed_folder)
!mkdir -p $homer_dir

### First extract upstream/downstream intron sequences

In [6]:
import os
import glob
import pybedtools

nt = 400

directions = 'upstream', 'downstream'

for bedfile in glob.iglob('{}/exon2*.bed'.format(bed_folder)):
    if 'stream' in bedfile:
        continue
    
    basename = os.path.basename(bedfile)
    print basename
    prefix = basename.split('.bed')[0]
    bed = pybedtools.BedTool(bedfile)
    
    for direction in directions:
        # Get downstream intron
        if direction == 'downstream':
            intron = bed.flank(l=0, r=nt, s=True, g=pybedtools.chromsizes('hg19'))
        elif direction == 'upstream':
            intron = bed.flank(l=nt, r=0, s=True, g=pybedtools.chromsizes('hg19'))
            
        # Saved every exon that was exactly upstream or downstream of a junction,
        # So when taking the flanking sequence, there's a lot of repetition
        unique = pybedtools.BedTool(list(set(x for x in intron)))


        print '\t', len(intron), len(unique)
        saveas = '{}/{}_{}{}.bed'.format(bed_folder, prefix, direction, nt)
        print '\t', saveas
        unique.saveas(saveas)
        ! wc -l $saveas

exon2_~0.bed
	1770 1770
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_upstream400.bed
1770 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_upstream400.bed
	1770 1770
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_downstream400.bed
1770 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_downstream400.bed
exon2_middle_background.bed
	8124 8124
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_middle_background_upstream400.bed
8124 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_middle_background_upstream400.bed
	8124 8124
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_middle_background_downstream400.bed
8124 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_middle_background_downstream400.bed
exon2_MN_bimodal_background.bed
	2206 2206
	/projects/ps-yeolab/obotvinnik/singlecell_pnms/figu

## Enrichment of within-phenotype modality

#### Background: All other modalities, within that phenotype

In [7]:
from modish import MODALITY_ORDER

In [8]:
import os
import fileinput

from gscripts.qtools import Submitter
import glob
import itertools

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6 -mset vertebrates -mis 0 -p {} -noweight'.format(n_processors)

commands = []


for bedfile in glob.iglob('{}/exon2*.bed'.format(bed_folder)):
    
    basename = os.path.basename(bedfile)
    if 'background' in basename:
        continue
    
    prefix = basename.split('.')[0]
    modality = prefix.split('_')[1]
    if modality not in MODALITY_ORDER:
        modality = prefix.split('_')[2]
#     print modality
    
    background = bedfile.replace(modality, modality + '_background')
    
    out_dir = '{}/{}'.format(homer_dir, basename.replace('.bed', ''))
#     if os.path.exists('{}/homerResults.html'):
#         print 'Already finished {}, moving on'.format(basename)
#         continue
    command = '{} {} hg19 {} -bg {} {}'.format(
        findMotifsGenome, bedfile, out_dir, background, homer_flags)
    print '\n', command
    commands.append(command)
#     ! $command
jobname = 'homer'
sub = Submitter(commands, jobname, walltime='0:30:00', queue='home-scrm',
               ppn=n_processors, write_and_submit=True, array=True,
               out_filename='{}/{}.out'.format(homer_dir, jobname),
               err_filename='{}/{}.err'.format(homer_dir, jobname),)

running 85 tasks as an array-job.
job ID: 3798111



/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/homer/exon2_~0 -bg /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_background.bed -rna -len 4,5,6 -mset vertebrates -mis 0 -p 4 -noweight

/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_upstream400.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/homer/exon2_~0_upstream400 -bg /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_~0_background_upstream400.bed -rna -len 4,5,6 -mset vertebrates -mis 0 -p 4 -noweight

/home/yeo-lab/software/homer/bin/findMotifsGenome.pl /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/exon2_NPC_~0_downstream200.bed hg19 /projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/home